In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [3]:
spark = SparkSession\
    .builder\
        .appName('Spotify_bronze')\
            .config("spark.sql.maxDepth", 10000) \
                .getOrCreate()

23/10/13 11:24:06 WARN Utils: Your hostname, GabGalani resolves to a loopback address: 127.0.1.1; using 192.168.230.128 instead (on interface ens33)
23/10/13 11:24:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/13 11:24:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# def print_full_schema(df, prefix=""):
#     for field in df.schema.fields:
#         if isinstance(field.dataType, f.StructType):
#             print(f"{prefix}{field.name}: struct")
#             print_full_schema(df.select(field.name), prefix + "  ")
#         else:
#             print(f"{prefix}{field.name}: {field.dataType}")

In [5]:
dataset = spark.read.json("/home/gabrielgalani/Documents/Airflow/Datalake/Bronze/Spotify")
dataset.cache()

DataFrame[tracks: struct<items:array<struct<album:struct<album_type:string,artists:array<struct<external_urls:struct<spotify:string>,href:string,id:string,name:string,type:string,uri:string>>,external_urls:struct<spotify:string>,href:string,id:string,images:array<struct<height:bigint,url:string,width:bigint>>,is_playable:boolean,name:string,release_date:string,release_date_precision:string,total_tracks:bigint,type:string,uri:string>,artists:array<struct<external_urls:struct<spotify:string>,href:string,id:string,name:string,type:string,uri:string>>,disc_number:bigint,duration_ms:bigint,explicit:boolean,external_ids:struct<isrc:string>,external_urls:struct<spotify:string>,href:string,id:string,is_local:boolean,is_playable:boolean,name:string,popularity:bigint,preview_url:string,track_number:bigint,type:string,uri:string>>>, extract_date: string]

In [6]:
# dataset.printSchema()

In [7]:
# ##exploded_items = dataset.select(f.explode(f.col("tracks.items")).alias("item"))
# dataset.select(f.explode('tracks.items').alias('items')).printSchema()

In [8]:
# dataset.select(f.explode('tracks.items.album').alias('items')).printSchema()

In [15]:
df_album = dataset.select(f.explode('tracks.items').alias('items'))\
.select(
        f.col('items.album.artists')[0]['id'].alias('album_id'),
        f.col('items.album.artists')[0]['name'].alias('album_name'),
        f.col('items.album.artists')[0]['type'].alias('album_type'),
        f.col('items.id').alias('music_id'),
        f.col('items.name').alias('music_name'),
        f.col('items.popularity').alias('music_popularity'),
        f.col('items.type').alias('music_type'),
        f.col('items.disc_number').alias('disc_number'), 
        'items.duration_ms',
        'items.explicit',
        'items.external_urls.spotify',
        'items.album.name',
        'items.album.id',
        'items.album.is_playable', 
        'items.album.album_type',
        f.col('items.artists.id')[0].alias('artist_id'),
        f.col('items.artists.name')[0].alias('artist_id'),
        f.col('items.artists.type')[0].alias('artist_id'),
        f.col('items.artists.id')[0].alias('artist_id')
        ).printSchema()
    

root
 |-- album_id: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- album_type: string (nullable = true)
 |-- music_id: string (nullable = true)
 |-- music_name: string (nullable = true)
 |-- music_popularity: long (nullable = true)
 |-- music_type: string (nullable = true)
 |-- disc_number: long (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- spotify: string (nullable = true)
 |-- name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- is_playable: boolean (nullable = true)
 |-- album_type: string (nullable = true)
 |-- artist_id: string (nullable = true)



In [32]:

df_album = dataset.select(f.explode('tracks.items').alias('items')) \
.select(
    f.col('items.album.artists')[0]['id'].alias('artist_of_the_publication_id'),
    f.col('items.album.artists')[0]['name'].alias('artist_of_the_publication'),
    f.col('items.album.artists')[0]['type'].alias('type_artist_of_the_publication'),
    f.col('items.album.artists')[0]['href'].alias('href_artist_of_the_publication'),
    f.col('items.id').alias('music_id'),
    f.col('items.name').alias('music_name'),
    f.col('items.popularity').alias('music_popularity'),
    f.col('items.type').alias('music_type'),
    f.col('items.disc_number').alias('disc_number'),
    f.col('items.track_number').alias('track_number'),
    'items.duration_ms',
    'items.explicit',
    'items.external_urls.spotify',
    f.col('items.album.name').alias('music/playlist_name'),
    f.col('items.album.id').alias('music/playlist_id'),
    f.col('items.album.is_playable').alias('music/playlist_is_playable'),
    f.col('items.album.album_type').alias('music/playlist_type'),
    f.col('items.album.release_date').alias('music/playlist_release_date'),
    f.col('items.album.total_tracks').alias('music/playlist_total_tracks'),
    f.explode('items.artists').alias('artist')
).select(
    'artist.id',
    'artist.name',
    'artist.type',
    f.col('artist.external_urls.spotify').alias('link_artist'),
    'artist_of_the_publication_id',
    'artist_of_the_publication',
    'type_artist_of_the_publication',
    'href_artist_of_the_publication',
    'music_id',
    'music_name',
    'track_number',
    'music_popularity',
    'music_type',
    'disc_number',
    'duration_ms',
    'explicit', 
    'spotify',
    'music/playlist_id',
    'music/playlist_name',
    'music/playlist_is_playable',
    'music/playlist_type',
    'music/playlist_release_date',
    'music/playlist_total_tracks'
)

df_album.show()

+--------------------+----------------+------+--------------------+----------------------------+-------------------------+------------------------------+------------------------------+--------------------+--------------------+------------+----------------+----------+-----------+-----------+--------+--------------------+--------------------+--------------------+--------------------------+-------------------+---------------------------+---------------------------+
|                  id|            name|  type|         link_artist|artist_of_the_publication_id|artist_of_the_publication|type_artist_of_the_publication|href_artist_of_the_publication|            music_id|          music_name|track_number|music_popularity|music_type|disc_number|duration_ms|explicit|             spotify|   music/playlist_id| music/playlist_name|music/playlist_is_playable|music/playlist_type|music/playlist_release_date|music/playlist_total_tracks|
+--------------------+----------------+------+--------------------

In [24]:
df_album = dataset.select(f.explode('tracks.items.album').alias('items'))\
    .select('items.album_type', 'items.href', 'items.id', 'items.name', 
            'items.release_date', 'items.release_date_precision', 
            'items.total_tracks', 'items.type', 'items.uri', 
            f.col('items.artists')[0]['id'].alias('artist_id'),
            f.col('items.artists')[0]['name'].alias('artist_name'),
            f.col('items.artists')[0]['type'].alias('artist_type'),
            f.col('items.images')[0]['url'].alias('album_image')
            ).printSchema()

root
 |-- album_type: string (nullable = true)
 |-- href: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- release_date_precision: string (nullable = true)
 |-- total_tracks: long (nullable = true)
 |-- type: string (nullable = true)
 |-- uri: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_type: string (nullable = true)
 |-- album_image: string (nullable = true)



In [111]:
df_album.show(20)

+----------+--------------------+--------------------+--------------------+------------+----------------------+------------+-----+--------------------+--------------------+-------------------+-----------+--------------------+
|album_type|                href|                  id|                name|release_date|release_date_precision|total_tracks| type|                 uri|           artist_id|        artist_name|artist_type|         album_image|
+----------+--------------------+--------------------+--------------------+------------+----------------------+------------+-----+--------------------+--------------------+-------------------+-----------+--------------------+
|    single|https://api.spoti...|59qldBzRiQutEYO1h...|Faz um Vuk Vuk (T...|  2023-07-20|                   day|           1|album|spotify:album:59q...|2UMj7NCbuqy1yUZmi...|   MC Kevin o Chris|     artist|https://i.scdn.co...|
|    single|https://api.spoti...|787d22Rcq1Rndn8D3...|Solteiro Forçado ...|  2023-07-07|        

In [112]:
df_album.coalesce(1).write.mode('overwrite').json('output/spotify')